# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [6]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [70]:
df.head(10)

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...
5,L'Or Espresso Café \n,Je trouve le goût médiocre. La mousse ne tient...
6,Hervidor de Agua Eléctrico,"Está lu bonita calienta muy rápido, es muy fun..."


## LLMChain

In [ ]:
!pip install langchain_community

In [8]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [10]:
#Replace None by your own value and justify
llm = ChatOpenAI(temperature=0)


In [22]:
prompt = ChatPromptTemplate.from_template( "descirbe this product: {product}")

In [23]:

chain = LLMChain(llm=llm, prompt=prompt)

In [24]:
product = 'knife'
chain.invoke(product)

{'product': 'knife',
 'text': 'A knife is a versatile tool with a sharp blade used for cutting, slicing, and chopping various items. It typically has a handle for gripping and a pointed, cutting edge. Knives come in various shapes and sizes, with different designs catering to specific purposes such as cooking, hunting, and survival. They are an essential tool in any kitchen and can also be used for outdoor activities, crafting, and self-defense.'}

## SimpleSequentialChain

In [20]:
from langchain.chains import SimpleSequentialChain

In [28]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "explain this item {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [31]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "answer, if this is a dangerous item: {first_prompt}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [32]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [ ]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
A knife is a tool used for cutting or slicing food, objects, or materials. It typically consists of a sharp blade attached to a handle. Knives come in various shapes and sizes, with different blade designs depending on their intended use. They can be made from a variety of materials such as stainless steel, carbon steel, ceramic, or titanium. Knives are commonly used in cooking, hunting, camping, and everyday tasks that require cutting or slicing.
Yes, a knife can be considered a dangerous item if not used properly. It has the potential to cause harm or injury if mishandled or used inappropriately. It is important to always exercise caution and follow safety guidelines when using a knife.

> Finished chain.


'Yes, a knife can be considered a dangerous item if not used properly. It has the potential to cause harm or injury if mishandled or used inappropriately. It is important to always exercise caution and follow safety guidelines when using a knife.'

**Repeat the above twice for different products**

## SequentialChain

In [55]:
from langchain.chains import SequentialChain

In [73]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "translate the review into spanish {review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key = "spanish_output"
                    )


second_prompt = ChatPromptTemplate.from_template(
    "summarize the review: {spanish_output}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key= "sp_summary_output"
                    )


# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "translate to english {sp_summary_output}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key= "english_output"
                      )



# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
        "display both {sp_summary_output} and {english_output}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key= "final_output"
                     )




overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables= ["review"],
    output_variables=["spanish_output", "sp_summary_output", "english_output", "final_output" ],
    verbose=True
)

review = df.Review[5]

result = overall_chain({"review": review})
result["final_output"]



> Entering new SequentialChain chain...

> Finished chain.


'El crítico encuentra que el sabor de la bebida es mediocre y la espuma no se mantiene, lo cual le parece extraño ya que ha comprado el mismo producto en la tienda y el sabor es mucho mejor. Se cuestiona si se trata de un lote antiguo o una falsificación.\n\nThe critic finds that the taste of the drink is mediocre and the foam does not hold, which seems strange to him since he has bought the same product in the store and the taste is much better. He questions if it is an old batch or a counterfeit.'

**Repeat the above twice for different products or reviews**

## Router Chain

In [ ]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [ ]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [ ]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain,RouterOutputParser
from langchain.prompts import PromptTemplate

In [ ]:
llm = ChatOpenAI(temperature=0)

In [ ]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [ ]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [ ]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [ ]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [ ]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

In [ ]:
chain.run("What is black body radiation?")

In [ ]:
chain.run("what is 2 + 2")

In [ ]:
chain.run("Why does every cell in our body contain DNA?")

**Repeat the above at least once for different inputs and chains executions - Be creative!**